# Workshop DL02: Convolutional Neural Networks*

## Agenda:
- Introduction to CNN   
- Apply CNN to MNIST dataset

Last week we have covered how to calssfy the MNIST digits using DNN and achieved roughly 95% accuracy. This weel we will try CNN and will achieve 99% accuracy! 


## Exercies:
- something

*We recommand running this notebook on colab, gcloud or other cloud instance to avoid over-heating you computer.

### CNN structure:

something





#### step 1: preparing the data
The ways we prepare the data for DNN and CNN are different due to the difference in structure. 


